In [1]:
import os
import cv2
import numpy as np
import sqlite3
from PIL import Image

# Face Recognition Project

This project demonstrates how to create a face recognition system using OpenCV and SQLite. The project is divided into three main parts: dataset creation, model training, and face detection.

## Prerequisites

- Python 3.x
- OpenCV
- NumPy
- Pillow
- SQLite3

Install the required Python packages using:

pip install opencv-python numpy pillow

## For clasifier ypu need to create a folder named as haarcascate and after that have to download from github link to the file [here](https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml) after download that file put it into the folder

## For DataFile creation 
Have to open [this link ](url_here) to create a table with this command 

Create table people (
  id INT,
  name varchar(50),
  age INT,
  gender varchar(10)
)

## After creation of DB file download it and past it at project 
The Default name is sqlite.db and if this name is not changed can run the script otherwise you have to replace it manually for all cells when is used.

# Dataset creation

In [25]:
faceDetect=cv2.CascadeClassifier('haarcascade/haarcascade_frontalface_default.xml');
cam=cv2.VideoCapture(0);

def insertOrUpdate(Id,Name,Age,Gen):
    conn=sqlite3.connect("sqlite.db")
    cmd="SELECT * FROM People WHERE ID="+str(Id)
    cursor=conn.execute(cmd)
    isRecordExist=0
    for row in cursor:
        isRecordExist=1
    if(isRecordExist==1):
        conn.execute("UPDATE People SET Name=? WHERE id=?", (Name,Id,))
        conn.execute("UPDATE People SET Age=? WHERE id=?",(Age, Id))
        conn.execute("UPDATE People SET Gender=? WHERE id=?", (Gen,Id,))
    else:
        conn.execute("INSERT INTO people(id,Name,Age,Gender) Values(?,?,?,?)", (Id, Name, Age, Gen))
    conn.commit()
    conn.close()

Id=input('Enter User Id:')
name=input('Enter User Name:')
age=input('Enter User Age:')
gen=input('Enter User Gender:')
insertOrUpdate(Id,name,age,gen)
sampleNum=0
while(True):
    ret,img=cam.read();
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=faceDetect.detectMultiScale(gray,1.3,5);
    for(x,y,w,h) in faces:
        sampleNum=sampleNum+1;
        cv2.imwrite("dataset/User."+str(Id)+"."+str(sampleNum)+".jpg",gray[y:y+h,x:x+w])
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.waitKey(400);
    cv2.imshow("Face",img);
    cv2.waitKey(1);
    if(sampleNum>20):
        break;
cam.release()
cv2.destroyAllWindows()

Enter User Id: 3
Enter User Name: Aydun Aliev
Enter User Age: 33
Enter User Gender: make


# Training Model

In [27]:
recognizer = cv2.face.LBPHFaceRecognizer_create()
path = "dataset"

def get_images_with_ids(path):
    image_paths = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.jpg') or f.endswith('.png')]
    faces = []
    ids = []
    
    for single_image_path in image_paths:
        try:
            face_img = Image.open(single_image_path).convert("L")  # Convert to grayscale
            face_np = np.array(face_img, np.uint8)
            id = int(os.path.split(single_image_path)[-1].split(".")[1])
            print(id)
            faces.append(face_np)
            ids.append(id)
            cv2.imshow("training", face_np)
            cv2.waitKey(10)
        except Exception as e:
            print(f"Error processing {single_image_path}: {e}")
            continue

    return np.array(ids), faces

# Create directory if it doesn't exist
output_dir = "trainingDataModel"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Train the recognizeModel
ids, faces = get_images_with_ids(path)
recognizer.train(faces, ids)
recognizer.save(os.path.join(output_dir, "trainingdata.yml"))
cv2.destroyAllWindows()

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


# Detect Faces

In [ ]:
facedetect = cv2.CascadeClassifier("haarcascade//haarcascade_frontalface_default.xml")
cam = cv2.VideoCapture(0)
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("trainingDataModel/trainingdata.yml")


def getProfile(id):
    conn=sqlite3.connect("sqlite.db")
    cursor=conn.execute("SELECT * FROM People WHERE id=?", (id,))
    profile=None
    for row in cursor:
        profile=row
    conn.close()
    return profile


while(True):
    ret,img=cam.read();
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=facedetect.detectMultiScale(gray,1.3,5);
    for(x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        id,conf=recognizer.predict(gray[y:y+h,x:x+w])
        profile=getProfile(id)
        print(profile)
        if(profile != None):
            cv2.putText(img, "Name : "+str(profile[1]), (x,y+h+20),cv2.FONT_HERSHEY_COMPLEX, 1,(0,255,127),2)
            cv2.putText(img, "Age : "+str(profile[2]), (x,y+h+45),cv2.FONT_HERSHEY_COMPLEX, 1,(0,255,127),2)
            cv2.putText(img, "Gender : "+str(profile[3]), (x,y+h+70),cv2.FONT_HERSHEY_COMPLEX, 1,(0,255,127),2)


    cv2.imshow("Face",img);
    if(cv2.waitKey(1)==ord('q')):
        break;

cam.release()
cv2.destroyAllWindows()


(3, 'Aydun Aliev', 33, 'make')
(3, 'Aydun Aliev', 33, 'make')
(3, 'Aydun Aliev', 33, 'make')
(3, 'Aydun Aliev', 33, 'make')
(3, 'Aydun Aliev', 33, 'make')
(1, 'Stefan Nikolov', 28, 'male')
(3, 'Aydun Aliev', 33, 'make')
(3, 'Aydun Aliev', 33, 'make')
(3, 'Aydun Aliev', 33, 'make')
(3, 'Aydun Aliev', 33, 'make')
(2, 'Mariq Blajeva', 23, 'female')
(2, 'Mariq Blajeva', 23, 'female')
(2, 'Mariq Blajeva', 23, 'female')
(3, 'Aydun Aliev', 33, 'make')
(2, 'Mariq Blajeva', 23, 'female')
(3, 'Aydun Aliev', 33, 'make')
(3, 'Aydun Aliev', 33, 'make')
(2, 'Mariq Blajeva', 23, 'female')
(2, 'Mariq Blajeva', 23, 'female')
(2, 'Mariq Blajeva', 23, 'female')
(3, 'Aydun Aliev', 33, 'make')
(2, 'Mariq Blajeva', 23, 'female')
(3, 'Aydun Aliev', 33, 'make')
(2, 'Mariq Blajeva', 23, 'female')
(3, 'Aydun Aliev', 33, 'make')
(2, 'Mariq Blajeva', 23, 'female')
(3, 'Aydun Aliev', 33, 'make')
(2, 'Mariq Blajeva', 23, 'female')
(3, 'Aydun Aliev', 33, 'make')
(1, 'Stefan Nikolov', 28, 'male')
(3, 'Aydun Aliev', 3